#Segmenting and Clustering Neighborhoods in Toronto

Getting dataframe from WikiPedia page using the pandas read_html

In [0]:
#Importing necessary libraris

import pandas as pd
import numpy as np
import csv

In [0]:
#reading the data. the na values are already considered as Not Assigned hence they will be not included

html=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', na_values='Not assigned', flavor='html5lib')
print(type(html[0]))
df=html[0]

<class 'pandas.core.frame.DataFrame'>


In [0]:
#check the dataframe
df.columns

Index(['Postal code', 'Borough', 'Neighborhood'], dtype='object')

In [0]:
#check the data
df.head()

,Postal code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [0]:
#dealing with NaN 
print(df.isnull().sum())
df.drop(df.index[0], inplace = True)
df.dropna(axis = 0, inplace = True)
print(df.isnull().sum())


Postal code     0
Borough         0
Neighborhood    0
dtype: int64
Postal code     0
Borough         0
Neighborhood    0
dtype: int64


(101, 3)

In [0]:
#the number of rows in the dataframe
df.shape

(101, 3)